#### Reading instance and defining model

In [839]:
# Reading instance

from instanceReaderSMDHPDPTW import Instance

instanceName = "AA10 - Modificada"

n_pickup_sites = 2

fleet_capacities = [10, 20]

vehicle_locations= [[0], [1]]

inst = Instance(instanceName,
                fleet_capacities,
                n_pickup_sites,
                multiple_depots=True,
                vehicle_locations=[[0], [1]])

    solver failure.


In [840]:
# Applying modifications - These would be more suitable on reader!
# but for now, I'll do it like that

# Instance data

df_instance = inst.instance_data

df_instance = df_instance[['type','x','y','d','w_a','w_b']]

# Taking away artificial nodes

valid_types = ['Depot', 'Pickup','Delivery']

df_instance = df_instance[df_instance['type'].isin(valid_types)].reset_index(drop=True)

# Agreggating differente pickup sites

# Storing demands

# pickup_demands = df_instance[df_instance['type'] == 'Pickup'].groupby('x')[['type','x','y','d']].agg({'d':'sum'}).reset_index()

# Dropping duplicate pickup sites

df_instance = df_instance.drop_duplicates(['type','x']).reset_index(drop=True)

# Multiplying demands by factor

pickup_factor = 10

delivery_factor = 0.5

df_instance['d'] = [demand*pickup_factor if (demand > 0) else demand*delivery_factor for demand in df_instance['d']]

# Appending back depot nodes at the end

df_instance = df_instance.append(df_instance[df_instance['type'] == 'Depot']).reset_index(drop=True)

In [841]:
# Distances

c = [[0 for i in range(len(df_instance))] for j in range(len(df_instance))]

for i in range(len(df_instance)):
    for j in range(len(df_instance)):
        
        x_i = df_instance.iloc[i]['x']
        
        x_j = df_instance.iloc[j]['x']
        
        y_i = df_instance.iloc[i]['y']
        
        y_j = df_instance.iloc[j]['y']
        
        # Calculando distância euclidiana, arrendondando para 2 casas
        dist = round(((x_i-x_j)**2 + (y_i-y_j)**2)**(1/2),2)
        c[i][j] = dist
        
# Demands

d = df_instance['d']

# Time windows

w_a = df_instance['w_a']

w_b = df_instance['w_b']


# Capacities

Q = fleet_capacities

# Depot of vehicle k

D = []

i = 0

for location in vehicle_locations:
    
    for vehicle in location:
        
        D.append(i)
    
    i += 1


In [842]:
# Importing pyomo modules

import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Creating model
model = pyo.ConcreteModel()

In [843]:
# Visualizing instance


df_instance.set_index(pd.Index(range(1, len(df_instance) + 1)))

,type,x,y,d,w_a,w_b
1,Depot,19.942,16.269,0.0,0.0,1000.0
2,Depot,8.195,24.765,0.0,0.0,1000.0
3,Pickup,19.942,16.269,70.0,0.0,1000.0
4,Pickup,8.195,24.765,50.0,0.0,1000.0
5,Delivery,35.573,46.733,-3.5,209.0,269.0
6,Delivery,36.158,11.897,-4.5,562.0,622.0
7,Delivery,43.760,46.624,-7.5,261.0,321.0
8,Delivery,13.387,36.048,-2.5,549.0,609.0
9,Delivery,21.442,34.131,-6.0,470.0,530.0
10,Delivery,4.515,24.464,-2.5,439.0,499.0


#### Defining sets and notation

#### Nodes sets

## $  N = S_{0} \cup P \cup D \cup S_{f} = \{1, ..., 3s + n\} $
## $ S_{0} = \{1, ... , s\} $
## $ P = \{s + 1, ... , 2s\} $
## $ D = \{2s + 1, ... , 2s + n\} $
## $ S_{f} = \{2s + n + 1, ... , 3s + n\} $

#### Vehicles set

## $ \newline K = \{1, ..., m\} $

#### Visits set

## $ \newline V = \{1, ..., MV\} $


In [844]:
# Sets

model.setN = pyo.Set(initialize=range(len(df_instance)))

model.setP = pyo.Set(initialize=df_instance[df_instance['type'] == 'Pickup'].index)

model.setD = pyo.Set(initialize=df_instance[df_instance['type'] == 'Delivery'].index)

# Additional sets

number_of_depots = int(len(df_instance[df_instance['type'] == 'Depot'])/2)

model.setS = pyo.Set(initialize=df_instance[df_instance['type'] == 'Depot'].index)

model.setS_0 = pyo.Set(initialize=df_instance[df_instance['type'] == 'Depot'].iloc[:number_of_depots].index)

model.setS_f = pyo.Set(initialize=df_instance[df_instance['type'] == 'Depot'].iloc[-number_of_depots:].index)

# Set of vehicles

model.setK = pyo.Set(initialize=range(len(fleet_capacities)))

# Set of visits

MV = 6

model.setV = pyo.Set(initialize=range(MV))

#### Defining variables

### $x_{ijkv} = \text{1 if vehicle $k$ travels from $i$ to $j$ in its $v$ visit and 0 otherwise;} $

### $y_{ikv} = \text{amount of load in vehicle $k$ after visiting node $i$ in its $v$ visit;} $

### $w_{ikv} = \text{arrival time of vehicle $k$ in node $i$ in its $v$ visit;} $

### $z_{ikv} = \text{amount of load supplied/delivered by vehicle $k$ to node $i$ in its $v$ visit;} $

In [845]:
# Variables

model.x = pyo.Var(model.setN, model.setN, model.setK, model.setV, within = pyo.Binary)

model.y = pyo.Var(model.setN, model.setK, model.setV, within = NonNegativeReals)

model.w = pyo.Var(model.setN, model.setK, model.setV, within = NonNegativeReals)

model.z = pyo.Var(model.setN, model.setK, model.setV, within = NonNegativeReals)

#### Defining constraints

#### Objective function

## $ (1) $

## $$ max \displaystyle \sum_{i \in D}\displaystyle \sum_{i \in K}\displaystyle \sum_{j \in V}z_{ikv}$$

In [846]:
# (1) - Objective function

model.exprobj = sum(sum(sum(model.z[i,k,v] for i in model.setD) for k in model.setK) for v in model.setV)

model.obj = pyo.Objective(expr= model.exprobj, sense = pyo.maximize)

## $ (2) $

## $$\displaystyle \sum_{i \in N}\displaystyle \sum_{v \in V}x_{ijkv} \leq \frac{MV}{2}, \hspace{15pt} \forall j \in P \cup D, k \in K$$

In [847]:
# (2) - Maximum number of visits in a node must be MV/2, which would be sucessive back and forth trips

model.R2 = pyo.ConstraintList()

for j in (model.setP | model.setD):
    
    for k in model.setK:
    
        model.R2.add(expr = sum(sum(model.x[i,j,k,v] for i in model.setN) for v in model.setV) <= MV/2)
    
        #model.R2.add(expr = sum(model.x[i,j] for i in (model.setN - model.setS_i) if i != j ) <= 1)

## $ (3) $

## $$\displaystyle \sum_{j \in N}\displaystyle \sum_{v \in V}x_{ijkv} \leq \frac{MV}{2}, \hspace{15pt} \forall i \in P \cup D, k \in K$$

In [848]:
# # Would this be required?

# # (3)

# model.R3 = pyo.ConstraintList()

# for i in (model.setP | model.setD):
    
#     for k in model.setK:
    
#         model.R3.add(expr = sum(sum(model.x[i,j,k,v] for j in model.setN) for v in model.setV) <= MV/2)
    
#         #model.R2.add(expr = sum(model.x[i,j] for i in (model.setN - model.setS_i) if i != j ) <= 1)

## $ (4) $


## $$ \displaystyle \sum_{i \in N \setminus S_{f}}x_{ihk(v-1)} -  \displaystyle \sum_{j \in N \setminus S_{0}}x_{hjkv} = 0, \hspace{15pt} \forall h \in P \cup D, k \in K, v \in V \setminus \{0\} $$

In [849]:
# (4) - Flow conservation - if vehicle k goes from i to h, it has to go from h to j on the next visit

model.R4 = pyo.ConstraintList()

for h in (model.setP | model.setD):
    
    for k in model.setK:
        
        for v in range(1,len(model.setV)):

            model.R4.add(expr = sum(model.x[i,h,k,v-1] for i in (model.setN - model.setS_f) if i != h) - sum(model.x[h,j,k,v] for j in (model.setN - model.setS_0) if j != h) == 0)

## $ (5) $

## $$ w_{jkv} \geq w_{ik(v-1)} + t_{ij} - M(1-x_{ijkv}), \newline\forall i \in N, j \in N, k \in K, v \in V \setminus\{0\} $$

In [850]:
# (5) - Time windows constraints - 1

# Defining big M

M = 1000

model.R5 = pyo.ConstraintList()

for i in model.setN:
    
    for j in model.setN:
        
        for k in model.setK:
    
            for v in range(1, len(model.setV)):
        
                model.R5.add(expr = model.w[j, k, v] >= model.w[i, k, v-1] + c[i][j] - M*(1 - model.x[i,j, k, v]))

## $ (6) $



## $$ w_{i}^{a} \leq w_{ikv} \leq w_{i}^{b} , \hspace{15pt} \forall i \in P \cup D, k \in K, v \in V $$

In [851]:
# (6)  - Time windows constraints - 2

model.R6 = pyo.ConstraintList()

for i in (model.setP | model.setD):
    
    for k in model.setK:
        
        for v in model.setV:
    
            model.R6.add(expr = model.w[i, k, v] >= w_a[i])
            model.R6.add(expr = model.w[i, k, v] <= w_b[i])
    

## $ (7) $



## $$ y_{jkv} \geq y_{ik(v-1)} + z_{jkv} - M(1-x_{ijkv}), \newline\forall i \in N, j \in P, k \in K, v \in V \setminus \{0\} $$

In [852]:
# (7) - Load constraints - 1 (pickup)

model.R7 = pyo.ConstraintList()

for i in model.setN:
    
    for j in model.setP:
        
        for k in model.setK:
    
            for v in range(1, len(model.setV)):
            #for v in range(len(model.setV)-1):
        
                model.R7.add(expr = model.y[j,k,v] >= model.y[i,k,v-1] + model.z[j,k,v] - M*(1 - model.x[i,j,k,v]))

## $ (8) $



## $$ y_{jkv} \geq y_{ik(v-1)} - z_{jkv} - M(1-x_{ijkv}), \newline\forall i \in N, j \in D, k \in K, v \in V \setminus \{0\} $$

In [853]:
# (8)  - Load constraints - 2 (delivery)

model.R8 = pyo.ConstraintList()

for i in model.setN:
    
    for j in model.setD:
        
        for k in model.setK:
    
            for v in range(1, len(model.setV)):
            #for v in range(len(model.setV)-1):
                
                #model.R7.add(expr = model.y[j,k,v+1] >= model.y[i,k,v] - model.z[j,k,v] - M*(1 - model.x[i,j,k,v+1]))
                model.R8.add(expr = model.y[j,k,v] >= model.y[i,k,v-1] - model.z[j,k,v] - M*(1 - model.x[i,j,k,v]))

## $ (9) $



## $$ \displaystyle y_{ikv} \leq Q_{k}, \hspace{15pt} \forall i \in N , k \in K, v \in V$$

In [854]:
# (9) - Load constraints - 3

model.R9 = pyo.ConstraintList()

for i in model.setN:
    
    #for j in model.setN:

    for k in model.setK:

        for v in model.setV:
                
                model.R9.add(expr = model.y[i,k,v] <= Q[k])

                #model.R9.add(expr = model.y[i,k,v] <= Q[k]*model.x[j,i,k,v])


## $ (10) $



## $$ \displaystyle x_{ijk0} = 0, \hspace{15pt} \forall i \in N \setminus S_{0} , j \in N , k \in K $$

In [855]:
# # (10) - Vehicles must depart from depot

# model.R10 = pyo.ConstraintList()

# for i in (model.setN - model.setS_0):
    
#     for j in model.setN:
    
#         for k in model.setK:
        
#             model.R10.add(expr = model.x[i,j,k,0] == 0)
            

## $ (11) $



## $$ \displaystyle x_{ijkMV} = 0, \hspace{15pt} \forall j \in N \setminus S_{f} , i \in N , k \in K $$

In [856]:
# (11) - Vehicles must end their routes in one of the available depots

model.R11 = pyo.ConstraintList()

for i in model.setN:
    
    for j in (model.setN - model.setS_f):
    
        for k in model.setK:
        
            model.R11.add(expr = model.x[i,j,k,model.setV.bounds()[1]] == 0)
            

## $ (12) $



## $$ \displaystyle x_{ijkv} = 0, \hspace{15pt} \forall i \in S_{0}, j \in D , k \in K, v \in V $$

In [857]:
# # (11) - Vehicles can't go from depot directly to a delivery node (was this supposed to be a constraint?)

# model.R12 = pyo.ConstraintList()

# for i in model.setS_0:
    
#     for j in model.setD:
    
#         for k in model.setK:
            
#             for v in model.setV:
        
#                 model.R12.add(expr = model.x[i,j,k,v] == 0)
            

## $ (13) $



## $$ \displaystyle \sum_{i \in N}\sum_{j \in N}\sum_{v \in V}x_{ijkv} = MV, \hspace{15pt} \forall k \in K$$

In [858]:
# (13) - Vehicles must visit exactly "MV" arcs in their routes

model.R13 = pyo.ConstraintList()
    
for k in model.setK:

        model.R13.add(expr = sum(sum(sum(model.x[i,j,k,v] for i in model.setN) for j in model.setN) for v in model.setV) == MV)

## $ (14) $

## $$ \displaystyle \sum_{j \in N}\sum_{k \in K}x_{ijk0} = K_{i}, \hspace{15pt} \forall i \in S_{0} $$

In [859]:
# (14) - Number of vehicles that depart from depot must be equal to number of vehicles available at depot

# K_{i} - Number of vehicles that start their route at depot i in S_0

K = [len(location) for location in vehicle_locations]

model.R14 = pyo.ConstraintList()

for i in (model.setS_0):

    model.R14.add(expr = sum(sum(model.x[i,j,k,0] for j in model.setN) for k in model.setK) == K[i])

## $ (15) $


## $$ \displaystyle x_{ijkMV} = 0, \hspace{15pt} \forall i \in P, j \in S_{f} , k \in K$$

In [860]:
# # (15) - Vehicles can't visit pickup node right before ending route (should be turned off in multiperiod formulation, maybe)

# model.R15 = pyo.ConstraintList()

# for i in model.setP:
    
#     for j in model.setS_f:
    
#         for k in model.setK:
        
#             model.R15.add(expr = model.x[i,j,k,model.setV.bounds()[1]] == 0)
            

## $ (16) $


## $$ \displaystyle \sum_{k \in K}\sum_{v \in V} z_{ikv} \leq d_{i}, \hspace{15pt} \forall i \in P \cup D $$

In [861]:
# (16)  - Total amount picked up/delivered to each node can't be above node supply/demand

model.R16 = pyo.ConstraintList()

for i in (model.setP | model.setD):

    model.R16.add(expr = sum(sum(model.z[i, k, v] for k in model.setK) for v in model.setV) <= abs(d[i]))

## $ (17) $


## $$  z_{ikv} \leq d_{i}\displaystyle \sum_{j \in N}x_{jikv}, \hspace{15pt} \forall i \in N, k \in K, v \in V $$

In [862]:
# (17) - Amount delivered to a node in a single visit can't exceed node demand (maybe redundant)

model.R17 = pyo.ConstraintList()

for i in model.setN:
    
    for k in model.setK:
        
        for v in model.setV:

            model.R17.add(expr = model.z[i, k, v] <= abs(d[i])*sum(model.x[j,i,k,v] for j in model.setN))

## $ (18) $


## $$  \displaystyle \sum_{i \in P}\sum_{k \in K}\sum_{v \in V} z_{ikv} =\displaystyle \sum_{i \in D}\sum_{k \in K}\sum_{v \in V}z_{ikv},$$

In [863]:
# (18) - Total delivered amount needs to be equal to total picked up demand

model.R18 = pyo.ConstraintList()

model.R18.add(expr = sum(sum(sum(model.z[i, k, v] for i in model.setP) for k in model.setK) for v in model.setV) == sum(sum(sum(model.z[i, k, v] for i in model.setD) for k in model.setK) for v in model.setV))

In [864]:
# # (19)  - Amount collected/delivered at node can't exceed vehicle capacity

# model.R19 = pyo.ConstraintList()

# for i in model.setN:
    
#     for k in model.setK:
        
#         for v in model.setV:

#             model.R19.add(expr = model.z[i,k,v] <= Q[k])

In [865]:
# # (20)

# model.R20 = pyo.ConstraintList()

# for i in model.setS_f:
    
#     for j in model.setN:
    
#         for k in model.setK:
            
#             for v in model.setV:
        
#                 model.R20.add(expr = model.x[i,j,k,v] == 0)
            

## $ (18) $


## $$ \displaystyle x_{ij} = 0,  \hspace{15pt} \forall i \in S; j \in P_{a} \setminus P_{i} $$

In [866]:
# # (21)

# # Validar melhor essa restrição depois!

# model.R21 = pyo.ConstraintList()

# for i in (model.setS_0):

#     for j in model.setN:
        
#         for k in model.setK:
            
#             if (k not in vehicle_locations[i]):
                       
#                 model.R21.add(expr = model.x[i, j, k, 0] == 0) 

#### Valid inequalities

In [867]:
# # Inequalities for N set

model.VI1 = pyo.ConstraintList()

    
# for i in model.setN:
for i in (model.setN -  model.setS_f):
    
    for k in model.setK:
        
        for v in model.setV:
    
            model.VI1.add(expr= model.x[i, i, k, v] == 0)

#             for h in model.setS_i:

#                 model.VI1.add(expr= model.x[i, h, k, v] == 0)

#             for h in model.setS_f:

#                 model.VI1.add(expr= model.x[h, i, k, v] == 0)
                
# for i in (model.setS_i | model.setS_f):
    
#     for j in (model.setS_i | model.setS_f):
        
#         for k in model.setK:
        
#             for v in model.setV:
    
#                 model.VI1.add(expr= model.x[i, j, k, v] == 0)

#### Solving model

In [868]:
# Solver

TimeLimit = 1200

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio201/cplex/bin/x64_win64/cplex')
opt.options['TimeLimit'] = TimeLimit


results = opt.solve(model, tee=True)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 20.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2020.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\User\AppData\Local\Temp\tmp5vgsxmvb.cplex.log' open.
CPLEX> New value for time limit in seconds: 1200
CPLEX> Problem 'C:\Users\User\AppData\Local\Temp\tmp18wedrod.pyomo.lp' read.
Read time = 0.03 sec. (0.70 ticks)
CPLEX> Problem name         : C:\Users\User\AppData\Local\Temp\tmp18wedrod.pyomo.lp
Objective sense      : Maximize
Variables            :    3649  [Nneg: 577,  Binary: 3072]
Objective nonzeros   :     120
Linear constraints   :    5930  [Less: 5044,  Greater: 144,  Equal: 742]
  Nonzeros           :   27801
  RHS nonzeros       :    4977

Variables            : Min LB: 0.00

In [869]:
for k in model.setK:
    
    for v in model.setV:

        for i in model.setN:

            for j in model.setN:
    
                try:
                    
                    if (pyo.value(model.x[i,j,k,v]))>0.00001:

                        print(f'x[{i}, {j}, {k}, {v}] == {pyo.value(model.x[i,j,k,v])}')
                    
                except:
                    
                    pass

x[1, 14, 0, 0] == 1.0
x[14, 9, 0, 0] == 1.0
x[15, 6, 0, 0] == 1.0
x[14, 10, 0, 2] == 1.0
x[14, 5, 0, 3] == 1.0
x[15, 13, 0, 3] == 1.0
x[0, 1, 1, 0] == 1.0
x[14, 4, 1, 0] == 1.0
x[14, 11, 1, 0] == 1.0
x[15, 2, 1, 0] == 0.9999997802197803
x[15, 8, 1, 0] == 1.0
x[14, 12, 1, 4] == 1.0


In [870]:
for k in model.setK:

    for v in model.setV:

        for i in model.setN:
            
            try:
                
                if pyo.value(model.z[i,k,v]) > 0.01:

                    print(f'z[{i}, {k}, {v}] == {pyo.value(model.z[i,k,v])}')
                    
            except:
                
                pass

z[6, 0, 0] == 7.5
z[9, 0, 0] == 2.5
z[10, 0, 2] == 5.0
z[5, 0, 3] == 4.5
z[13, 0, 3] == 6.0
z[2, 1, 0] == 44.50000054945055
z[4, 1, 0] == 3.5000007692307693
z[8, 1, 0] == 6.0
z[11, 1, 0] == 4.0
z[12, 1, 4] == 5.5


In [871]:
for k in model.setK:

    for v in model.setV:

        for i in model.setN:
            
            if pyo.value(model.y[i,k,v]) > 0.01:

                print(f'y[{i}, {k}, {v}] == {pyo.value(model.y[i,k,v])}')
                    

y[2, 0, 3] == 10.0
y[2, 0, 4] == 10.0
y[3, 0, 4] == 10.0
y[4, 0, 4] == 10.0
y[5, 0, 4] == 10.0
y[6, 0, 4] == 10.0
y[7, 0, 4] == 10.0
y[8, 0, 4] == 10.0
y[9, 0, 4] == 10.0
y[10, 0, 4] == 10.0
y[11, 0, 4] == 10.0
y[12, 0, 4] == 10.0
y[13, 0, 4] == 10.0
y[2, 0, 5] == 10.0
y[3, 0, 5] == 10.0
y[4, 0, 5] == 10.0
y[5, 0, 5] == 10.0
y[6, 0, 5] == 10.0
y[7, 0, 5] == 10.0
y[8, 0, 5] == 10.0
y[9, 0, 5] == 10.0
y[10, 0, 5] == 10.0
y[11, 0, 5] == 10.0
y[12, 0, 5] == 10.0
y[13, 0, 5] == 10.0
y[2, 1, 4] == 20.0
y[3, 1, 4] == 20.0
y[4, 1, 4] == 20.0
y[5, 1, 4] == 20.0
y[6, 1, 4] == 20.0
y[7, 1, 4] == 20.0
y[8, 1, 4] == 20.0
y[9, 1, 4] == 20.0
y[10, 1, 4] == 20.0
y[11, 1, 4] == 20.0
y[12, 1, 4] == 20.0
y[13, 1, 4] == 20.0
y[2, 1, 5] == 20.0
y[3, 1, 5] == 20.0
y[4, 1, 5] == 20.0
y[5, 1, 5] == 20.0
y[6, 1, 5] == 20.0
y[7, 1, 5] == 20.0
y[8, 1, 5] == 20.0
y[9, 1, 5] == 20.0
y[10, 1, 5] == 20.0
y[11, 1, 5] == 20.0
y[12, 1, 5] == 20.0
y[13, 1, 5] == 20.0


In [368]:
# # Inequalities for P set

# model.VI2 = pyo.ConstraintList()

# for i in model.setP:
    
#     for h in model.setS_i:
        
#         model.VI2.add(expr= model.x[h,inst.n + i] == 0)
        
#     for h in model.setS_f:
        
#         model.VI2.add(expr= model.x[i,h] == 0)
        
#     model.VI2.add(expr= model.x[inst.n + i, i] == 0)
    

In [369]:
# # Inequalities for D set

# model.VI3 = pyo.ConstraintList()

# for i in model.setD:
    
#     for h in model.setS_i:
    
#         model.VI3.add(expr= model.x[h, i] == 0)

In [370]:
# # Inequalities for P U D set

# model.VI4 = pyo.ConstraintList()

# for i in (model.setP | model.setD):
    
#     for j in (model.setP | model.setD):
        
#         if inst.w_a[i] + inst.c[i][j] > inst.w_b[j]:
    
#             model.VI4.add(expr= model.x[i, j] == 0)
    

In [371]:
# # Inequalities for artificial nodes (trying to improve)

# model.VI6 = pyo.ConstraintList()

# for i in (model.setP | model.setD):
    
#     for j in model.setP_a:
        
#         model.VI6.add(expr = model.x[i,j] == 0)
        
# model.VI7 = pyo.ConstraintList()

# for i in model.setD_a:
    
#     for j in (model.setP | model.setD):
        
#         model.VI7.add(expr = model.x[i,j] == 0)
        

In [372]:
# # Getting routes on solution

# routes = []

# # Start visiting nodes

# for i in model.setS_i:
    
#     for j in model.setN:

#         if pyo.value(model.x[i,j]) > 0.1:

#             routes.append([i, j])
        
# # Iterating for each route

# for route in routes:
    
#     last_node = route[-1]
    
#     while (last_node not in model.setS_f):

#         last_node = route[-1]
        
#         #print(last_node)

#         for j in model.setN:

#             if pyo.value(model.x[last_node,j]) > 0.1:

#                 route.append(j)


# print(routes)

In [373]:
# import plotly.express as px
# import plotly.graph_objects as go
# import matplotlib, random

# random.seed(123)
# colors = dict(matplotlib.colors.cnames.items())

# hex_colors = tuple(colors.values())


# fig = px.scatter(df_instance,x="x", y="y", symbol='type', symbol_sequence=["square","star","cross", "star", "x"], color='type', text='index', hover_data={"x":False, "y":False,  "d":True, "type":True, "index":True})

# fig.update_traces(textposition='top center')

# fig.update_layout(font=dict(size=9))

# # Create scatter trace of text labels

# for route in routes:
    
#     color = random.choice(hex_colors)
    
#     for index_node in range(1, len(route)):
        
#         fig.add_shape(type="line",
#             x0=df_instance.iloc[route[index_node-1]].x, 
#             y0=df_instance.iloc[route[index_node-1]].y,
#             x1=df_instance.iloc[route[index_node]].x,
#             y1=df_instance.iloc[route[index_node]].y,

#             line=dict(color=color,width=3), 
#             opacity = 0.2,
#             visible = True
#         )
        


# fig.show()


In [872]:
print("Hello world!")

Hello world!
